📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Load Images](#toc2_)    
- [Color Space Conversion](#toc3_)    
  - [RGB - Grayscale](#toc3_1_)    
    - [Manual](#toc3_1_1_)    
    - [Using OpenCV](#toc3_1_2_)    
    - [Using PIL](#toc3_1_3_)    
    - [Using Scikit-Image](#toc3_1_4_)    
  - [RGB - BGR](#toc3_2_)    
    - [Manual](#toc3_2_1_)    
    - [Using OpenCV](#toc3_2_2_)    
  - [RGB - YUV](#toc3_3_)    
    - [Manual](#toc3_3_1_)    
    - [Using OpenCV](#toc3_3_2_)    
  - [RGB - YCbCr](#toc3_4_)    
    - [Manual](#toc3_4_1_)    
    - [Using OpenCV](#toc3_4_2_)    
  - [RGB - HSV](#toc3_5_)    
    - [Using OpenCV](#toc3_5_1_)    
  - [RGB - CMYK](#toc3_6_)    
    - [Using PIL](#toc3_6_1_)    
- [Visualize using Custom Color Maps](#toc4_)    
  - [Indexed Image](#toc4_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from matplotlib.colors import ListedColormap
from numpy.typing import NDArray
from PIL import Image

In [ ]:
np.set_printoptions(linewidth=120)

# <a id='toc2_'></a>[Load Images](#toc0_)

In [ ]:
image_1 = Image.open(fp="../../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif")
image_2 = Image.open(fp="../../assets/images/misc/lenna_rgba_indexed.png")

# PIL.Image.Image to np.ndarray
im_1 = np.array(image_1)
im_2 = np.array(image_2)

# plot
fig, axs = plt.subplots(1, 2, figsize=(8, 4), layout="compressed")
axs[0].imshow(image_1, vmin=0, vmax=255)
axs[0].set_title("RGB")
axs[0].axis("off")
axs[1].imshow(image_2, vmin=0, vmax=255)
axs[1].set_title("Indexed RGBA")
axs[1].axis("off")
plt.show()

In [ ]:
# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["Original", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_1, im_1[:, :, 0], im_1[:, :, 1], im_1[:, :, 2]]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if len(img.shape) == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

# <a id='toc3_'></a>[Color Space Conversion](#toc0_)

📝 **Docs**:

- `cv2.cvtColor`: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14)
- Color Space Conversions [`cv2`]: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html)
- Modes [`PIL`]: [pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)
- Colormap reference [`matplotlib`]: [matplotlib.org/stable/gallery/color/colormap_reference.html](https://matplotlib.org/stable/gallery/color/colormap_reference.html)
- ITU-R Recommendation BT.601-7 [an international technical standard]: [itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf](http://itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf)

## <a id='toc3_1_'></a>[RGB - Grayscale](#toc0_)

### <a id='toc3_1_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_GS = np.array(
    [
        [0.299, 0.587, 0.114],
    ],
    dtype=np.float32,
)

# approximation using pseudo-inverse [best linear least-squares approximation]
GS_TO_RGB = np.linalg.pinv(RGB_TO_GS)

In [ ]:
def rgb_to_gs(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_GS.T).reshape(image.shape[:2])
    return np.clip(im, 0, 255).astype(np.uint8)


def gs_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    rgb = (image.reshape(-1, 1) @ GS_TO_RGB.T).reshape(*image.shape, 3)
    return np.clip(rgb, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_gs_1  = rgb_to_gs(im_1)
gs_1_to_rgb_1 = gs_to_rgb(im_1_to_gs_1)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_1, gs_1_to_rgb_1]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_gs_2 = cv2.cvtColor(im_1, cv2.COLOR_RGB2GRAY)

# replicates the grayscale channel into all three RGB channels (still grayish image)
gs_2_to_rgb_2 = cv2.cvtColor(im_1_to_gs_2, cv2.COLOR_GRAY2RGB)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_2, gs_2_to_rgb_2]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
im_1_to_gs_3 = Image.fromarray(im_1).convert("L")
gs_3_to_rgb_3 = im_1_to_gs_3.convert("RGB")

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_3, gs_3_to_rgb_3]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_1_4_'></a>[Using Scikit-Image](#toc0_)


In [ ]:
im_1_to_gs_4 = ski.color.rgb2gray(im_1)
gs_4_to_rgb_4 = ski.color.gray2rgb(im_1_to_gs_4)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_4, gs_4_to_rgb_4]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_2_'></a>[RGB - BGR](#toc0_)

### <a id='toc3_2_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_BGR = np.array(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=np.float32,
)

BGR_TO_RGB = np.linalg.inv(RGB_TO_BGR)

In [ ]:
def rgb_to_bgr(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_BGR.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def bgr_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ BGR_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_bgr = rgb_to_bgr(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_1_to_bgr] + [im_1_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_2_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_bgr = cv2.cvtColor(im_1, cv2.COLOR_RGB2BGR)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Blue Channel", "Green Channel", "Red Channel"]
images = [im_1_to_bgr] + [im_1_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_3_'></a>[RGB - YUV](#toc0_)

### <a id='toc3_3_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_YUV = np.array(
    [
        [0.29900, 0.58700, 0.11400],
        [-0.14713, -0.28886, 0.43600],
        [0.61500, -0.51499, -0.10001],
    ],
    dtype=np.float32,
)

YUV_TO_RGB = np.linalg.inv(RGB_TO_YUV)
YUV_OFFSET = np.array([0, 128, 128])

In [ ]:
def rgb_to_yuv(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YUV.T + YUV_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def yuv_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YUV_OFFSET) @ YUV_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_yuv = rgb_to_yuv(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_1_to_yuv] + [im_1_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_3_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_yuv = cv2.cvtColor(im_1, cv2.COLOR_RGB2YUV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_1_to_yuv] + [im_1_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_4_'></a>[RGB - YCbCr](#toc0_)

### <a id='toc3_4_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_YCBCR = np.array(
    [
        [0.299, 0.587, 0.114],
        [-0.168736, -0.331264, 0.5],
        [0.5, -0.418688, -0.081312],
    ],
    dtype=np.float32,
)

YCBCR_TO_RGB = np.linalg.inv(RGB_TO_YCBCR)
YCBCR_OFFSET = np.array([0, 128, 128])

In [ ]:
def rgb_to_ycbcr(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YCBCR.T + YCBCR_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def ycbcr_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YCBCR_OFFSET) @ YCBCR_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_ycrcb = rgb_to_ycbcr(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_1_to_ycrcb] + [im_1_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc3_4_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_ycrcb = cv2.cvtColor(im_1, cv2.COLOR_RGB2YCrCb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_1_to_ycrcb] + [im_1_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_5_'></a>[RGB - HSV](#toc0_)

### <a id='toc3_5_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_hsv = cv2.cvtColor(im_1, cv2.COLOR_RGB2HSV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["HSV Image", "Hue Channel", "Saturation Channel", "Value Channel"]
images = [im_1_to_hsv] + [im_1_to_hsv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc3_6_'></a>[RGB - CMYK](#toc0_)


### <a id='toc3_6_1_'></a>[Using PIL](#toc0_)


In [ ]:
im_1_to_cmyk = np.array(Image.fromarray(im_1).convert('CMYK'))


# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 4), layout="compressed")
titles = ["CMYK Image", "Cyan", "Magenta", "Yellow", "Key"]
images = [Image.fromarray(im_1).convert('CMYK')] + [im_1_to_cmyk[:, :, i] for i in range(4)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

# <a id='toc4_'></a>[Visualize using Custom Color Maps](#toc0_)


## <a id='toc4_1_'></a>[Indexed Image](#toc0_)


In [ ]:
# fetch color palletes
colors = np.array(image_2.getpalette(), dtype=np.uint8).reshape(-1, 3)
num_colors =colors.shape[0]

# find alpha index
alpha_idx = np.ones(num_colors)
if 'transparency' in image_2.info:
    t = image_2.info['transparency']
    if isinstance(t, bytes):
        alpha_idx = np.array(list(t), dtype=np.uint8) / 255.0
    elif isinstance(t, int):
        alpha_idx[t] = 0.0

# add alpha channel + normalization
colors_norm = np.concatenate([colors / 255.0, alpha_idx[:, None]], axis=1)

# log
print(f"num colors : {num_colors}")
print(f"alpha idx  : {alpha_idx}")
print(f"colors:\n{colors_norm}")

In [ ]:
# create a color map
cmap = ListedColormap(colors_norm)

# plot
fig, axs = plt.subplots(1, 2, figsize=(8, 4), layout="compressed")
axs[0].imshow(im_2, cmap='gray', vmin=0, vmax=im_2.max())
axs[0].set_title("Wrong cmap")
axs[0].axis("off")
axs[1].imshow(im_2, cmap=cmap, vmin=0, vmax=num_colors)
axs[1].set_title("Correct cmap")
axs[1].axis("off")
plt.show()